In [ ]:
from modules.feature_selector import (
    a_to_e_base_features,
    get_required_base_columns_for_derived_features,
    generate_common_derived_features
)
from modules.preprocess_utils import map_categorical_columns
from modules.feature_selector import generate_rfm_features_original, rfm_base_columns
import pandas as pd

# ✅ 파일 경로
file_path = "../../data/통합_train_데이터.parquet"


# ✅ 전체 불러올 컬럼 조합
base_cols = ["ID", "Segment"]
required_cols = get_required_base_columns_for_derived_features()
train_cols = sorted(set(base_cols + a_to_e_base_features + required_cols + rfm_base_columns))
train_df = pd.read_parquet(file_path, columns=train_cols)             # 데이터 로드

# ✅ 데이터 불러오기 + 전처리 파이프라인
train_df = map_categorical_columns(train_df)                          # 범주형 인코딩
train_df = generate_rfm_features_original(train_df)                   # RFM 변수 및 세그먼트 생성
train_df = generate_common_derived_features(train_df)                 # 파생변수 생성


[연회비발생카드수_B0M] 인코딩 완료
[한도증액횟수_R12M] 인코딩 완료
[이용금액대] 중간값 인코딩 완료
[할인건수_R3M] 인코딩 완료
[할인건수_B0M] 인코딩 완료
✔️ 파생변수 생성 완료.


In [6]:
train_df =generate_rfm_features(train_df)

⚠️ Recency 분위수 구간 생성 실패(Bin labels must be one fewer than the number of bin edges), 기본 점수 1 부여
⚠️ Frequency 분위수 구간 생성 실패(Bin labels must be one fewer than the number of bin edges), 기본 점수 1 부여
⚠️ Monetary 분위수 구간 생성 실패(Bin labels must be one fewer than the number of bin edges), 기본 점수 1 부여


In [5]:
missing_rfm = [col for col in rfm_base_columns if col not in train_df.columns]
print("❗ 누락된 RFM 컬럼:", missing_rfm)


❗ 누락된 RFM 컬럼: []


In [4]:
print("📦 rfm_base_columns 확인:", rfm_base_columns)


📦 rfm_base_columns 확인: ['입회경과개월수_신용', '이용건수_신용_R6M', '이용건수_체크_R6M', '이용금액_일시불_R6M', '이용금액_할부_R6M', '이용금액_CA_R6M', '이용금액_체크_R6M', '이용금액_카드론_R6M']


In [7]:
# 1. 컬럼 내 결측치 비율 체크
for col in rfm_base_columns:
    if col in train_df.columns:
        print(f"{col} 결측치 비율: {train_df[col].isna().mean():.3f}")

# 2. 컬럼 내 값 분포 체크 (특히 0과 음수 비율)
for col in rfm_base_columns:
    if col in train_df.columns:
        print(f"{col} 최소값: {train_df[col].min()}, 최대값: {train_df[col].max()}")
        print(f"{col} 0 값 비율: {(train_df[col] == 0).mean():.3f}")


입회경과개월수_신용 결측치 비율: 0.000
이용건수_신용_R6M 결측치 비율: 0.000
이용건수_체크_R6M 결측치 비율: 0.000
이용금액_일시불_R6M 결측치 비율: 0.000
이용금액_할부_R6M 결측치 비율: 0.000
이용금액_CA_R6M 결측치 비율: 0.000
이용금액_체크_R6M 결측치 비율: 0.000
이용금액_카드론_R6M 결측치 비율: 0.000
입회경과개월수_신용 최소값: 0, 최대값: 0
입회경과개월수_신용 0 값 비율: 1.000
이용건수_신용_R6M 최소값: 0, 최대값: 0
이용건수_신용_R6M 0 값 비율: 1.000
이용건수_체크_R6M 최소값: 0, 최대값: 0
이용건수_체크_R6M 0 값 비율: 1.000
이용금액_일시불_R6M 최소값: 0, 최대값: 0
이용금액_일시불_R6M 0 값 비율: 1.000
이용금액_할부_R6M 최소값: 0, 최대값: 0
이용금액_할부_R6M 0 값 비율: 1.000
이용금액_CA_R6M 최소값: 0, 최대값: 0
이용금액_CA_R6M 0 값 비율: 1.000
이용금액_체크_R6M 최소값: 0, 최대값: 0
이용금액_체크_R6M 0 값 비율: 1.000
이용금액_카드론_R6M 최소값: 0, 최대값: 0
이용금액_카드론_R6M 0 값 비율: 1.000


In [2]:
import pyarrow.parquet as pq

parquet_path = "../../data/통합_train_데이터.parquet"
schema = pq.read_schema(parquet_path)
all_columns = schema.names

# 확인
for col in rfm_base_columns:
    print(f"{col}: {'✅ 있음' if col in all_columns else '❌ 없음'}")


입회경과개월수_신용: ✅ 있음
이용건수_신용_R6M: ✅ 있음
이용건수_체크_R6M: ✅ 있음
이용금액_일시불_R6M: ✅ 있음
이용금액_할부_R6M: ✅ 있음
이용금액_CA_R6M: ✅ 있음
이용금액_체크_R6M: ✅ 있음
이용금액_카드론_R6M: ✅ 있음


In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def fast_vif(X, sample_size=5000, verbose=True):
    """
    빠르고 안전한 VIF 계산 함수
    - StandardScaler 적용
    - LinearRegression 기반
    - 고정값 제거 + 결측치 채움 + 샘플링 적용
    """
    X = X.copy()
    X = X.loc[:, X.nunique() > 1]  # 고정값 제거
    X = X.fillna(0)

    if X.shape[0] > sample_size:
        X = X.sample(n=sample_size, random_state=42)

    X_scaled = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns)
    vif_dict = {}

    for i in range(X_scaled.shape[1]):
        y = X_scaled.iloc[:, i]
        X_not_i = X_scaled.drop(X_scaled.columns[i], axis=1)
        model = LinearRegression().fit(X_not_i, y)
        r2 = model.score(X_not_i, y)
        vif = 1 / (1 - r2) if r2 < 1 else np.inf
        vif_dict[X_scaled.columns[i]] = vif
        if verbose:
            print(f"{X_scaled.columns[i]}: VIF={vif:.2f}")

    return pd.DataFrame(vif_dict.items(), columns=["feature", "VIF"]).sort_values(by="VIF", ascending=False)


In [7]:
# 파생변수 포함된 train_df에서 Segment, ID 제거
X = train_df.drop(columns=["ID", "Segment"])

# fast VIF 실행
vif_df = fast_vif(X, sample_size=5000)

# 결과 출력
print(vif_df.head(30))

이용건수_오프라인_B0M: VIF=37.91
잔액_신판ca최대한도소진율_r3m: VIF=inf
이용금액_오프라인_R6M: VIF=12.13
컨택건수_이용유도_청구서_R6M: VIF=2.98
이용건수_온라인_R6M: VIF=10.69
방문월수_PC_R6M: VIF=1.32
정상입금원금_B0M: VIF=inf
이용금액_일시불_R12M: VIF=29.26
이용금액_R3M_신용체크: VIF=17.29
최초한도금액: VIF=1.64
포인트_마일리지_환산_B0M: VIF=41.74
이용건수_일시불_R12M: VIF=181833.69
_1순위업종_이용금액: VIF=7.27
할인건수_R3M: VIF=2.43
포인트_포인트_건별_B0M: VIF=2.00
이용금액_오프라인_B0M: VIF=14.07
잔액_일시불_B0M: VIF=8.14
인입횟수_IB_R6M: VIF=16.49
할인건수_B0M: VIF=1.45
불만제기후경과월_R12M: VIF=1.58
승인거절건수_입력오류_R3M: VIF=inf
쇼핑_도소매_이용금액: VIF=4.77
증감율_이용건수_일시불_분기: VIF=25.42
RV최소결제비율: VIF=2.93
CA한도금액: VIF=8.24
이용건수_선결제_R6M: VIF=4.75
제휴연회비_B0M: VIF=4508965.30
방문횟수_앱_B0M: VIF=70.19
컨택건수_이용유도_인터넷_R6M: VIF=1.37
이용후경과월_신용: VIF=5.55
인입일수_ARS_B0M: VIF=1.88
거주시도명: VIF=1.11
청구금액_R6M: VIF=15.29
이용건수_페이_온라인_R6M: VIF=6.50
컨택건수_이용유도_청구서_B0M: VIF=3.06
평잔_일시불_해외_6M: VIF=7.44
최대이용금액_일시불_R12M: VIF=5.30
탈회횟수_누적: VIF=1.33
이용카드수_신용체크: VIF=8.44
이용후경과월_일시불: VIF=2.10
연체원금_최근: VIF=2.89
방문일수_앱_R6M: VIF=15.05
방문일수_앱_B0M: VIF=61.43
유효카드수_신용체크: VI

In [8]:
high_vif = vif_df[vif_df["VIF"] >= 10]
print("🔍 VIF ≥ 10 컬럼 수:", len(high_vif))
display(high_vif)

🔍 VIF ≥ 10 컬럼 수: 90


,feature,VIF
183,청구금액_단기증가율,inf
109,이용여부_3M_해외겸용_본인,inf
49,포인트_마일리지_월적립_B0M,inf
189,신판CA_소진율_장단기차,inf
57,정상청구원금_B2M,inf
...,...,...
4,이용건수_온라인_R6M,10.687885
61,한도증액금액_R12M,10.568046
139,한도증액횟수_R12M,10.510095
97,청구서발송여부_R3M,10.386427


In [ ]:
def drop_high_vif_features(df, vif_df, threshold=30.0, protected_features=None):
    """
    VIF ≥ threshold 컬럼 중 보호 피처 제외 후 제거
    """
    protected_features = protected_features or []
    
    # VIF 기준 제거 후보
    high_vif_df = vif_df[vif_df["VIF"] >= threshold]
    
    # 보호 피처 제외
    to_remove = [
        row["feature"] for _, row in high_vif_df.iterrows()
        if row["feature"] not in protected_features
    ]
    
    print(f"🚫 실제 제거될 고 VIF 컬럼 수 (보호 제외 후): {len(to_remove)}")
    print("🗂️ 제거되는 컬럼 목록:")
    for col in to_remove:
        print("-", col)
    
    df_reduced = df.drop(columns=to_remove)
    return df_reduced, to_remove


In [11]:
X = train_df.drop(columns=["ID", "Segment"])
vif_df = fast_vif(X)

# 주요 피처 보호
protected_features = [
    "정상청구원금_B0M", "정상청구원금_B5M", "청구금액_B0",
    "청구금액_R3M", "이용금액_R3M_신용", "이용금액_일시불_B0M",
    "잔액_신판ca최대한도소진율_r3m", "이용금액_R3M_신용체크"
]

# 고 VIF 제거 (보호 피처 제외)
X_reduced, removed_cols = drop_high_vif_features(X, vif_df, threshold=10.0, protected_features=protected_features)

# 최종 피처셋
abcd_final_features = X_reduced.columns.tolist()


이용건수_오프라인_B0M: VIF=37.91
잔액_신판ca최대한도소진율_r3m: VIF=inf
이용금액_오프라인_R6M: VIF=12.13
컨택건수_이용유도_청구서_R6M: VIF=2.98
이용건수_온라인_R6M: VIF=10.69
방문월수_PC_R6M: VIF=1.32
정상입금원금_B0M: VIF=inf
이용금액_일시불_R12M: VIF=29.26
이용금액_R3M_신용체크: VIF=17.29
최초한도금액: VIF=1.64
포인트_마일리지_환산_B0M: VIF=41.74
이용건수_일시불_R12M: VIF=181833.69
_1순위업종_이용금액: VIF=7.27
할인건수_R3M: VIF=2.43
포인트_포인트_건별_B0M: VIF=2.00
이용금액_오프라인_B0M: VIF=14.07
잔액_일시불_B0M: VIF=8.14
인입횟수_IB_R6M: VIF=16.49
할인건수_B0M: VIF=1.45
불만제기후경과월_R12M: VIF=1.58
승인거절건수_입력오류_R3M: VIF=inf
쇼핑_도소매_이용금액: VIF=4.77
증감율_이용건수_일시불_분기: VIF=25.42
RV최소결제비율: VIF=2.93
CA한도금액: VIF=8.24
이용건수_선결제_R6M: VIF=4.75
제휴연회비_B0M: VIF=4508965.30
방문횟수_앱_B0M: VIF=70.19
컨택건수_이용유도_인터넷_R6M: VIF=1.37
이용후경과월_신용: VIF=5.55
인입일수_ARS_B0M: VIF=1.88
거주시도명: VIF=1.11
청구금액_R6M: VIF=15.29
이용건수_페이_온라인_R6M: VIF=6.50
컨택건수_이용유도_청구서_B0M: VIF=3.06
평잔_일시불_해외_6M: VIF=7.44
최대이용금액_일시불_R12M: VIF=5.30
탈회횟수_누적: VIF=1.33
이용카드수_신용체크: VIF=8.44
이용후경과월_일시불: VIF=2.10
연체원금_최근: VIF=2.89
방문일수_앱_R6M: VIF=15.05
방문일수_앱_B0M: VIF=61.43
유효카드수_신용체크: VI

In [12]:
abcd_final_features = X_reduced.columns.tolist()

# 저장 (선택)
pd.Series(abcd_final_features).to_csv("abcd_final_features.csv", index=False)


In [13]:
abcd_final_features

['잔액_신판ca최대한도소진율_r3m',
 '컨택건수_이용유도_청구서_R6M',
 '방문월수_PC_R6M',
 '이용금액_R3M_신용체크',
 '최초한도금액',
 '_1순위업종_이용금액',
 '할인건수_R3M',
 '포인트_포인트_건별_B0M',
 '잔액_일시불_B0M',
 '할인건수_B0M',
 '불만제기후경과월_R12M',
 '쇼핑_도소매_이용금액',
 'RV최소결제비율',
 'CA한도금액',
 '이용건수_선결제_R6M',
 '컨택건수_이용유도_인터넷_R6M',
 '이용후경과월_신용',
 '인입일수_ARS_B0M',
 '거주시도명',
 '이용건수_페이_온라인_R6M',
 '컨택건수_이용유도_청구서_B0M',
 '평잔_일시불_해외_6M',
 '최대이용금액_일시불_R12M',
 '탈회횟수_누적',
 '이용카드수_신용체크',
 '이용후경과월_일시불',
 '연체원금_최근',
 '유효카드수_신용체크',
 '이용금액_일시불_B0M',
 '청구서발송여부_B0',
 '청구금액_R3M',
 '변동률_할부평잔',
 '회원여부_연체',
 '선입금원금_B2M',
 '포인트_이용포인트_R12M',
 '일시불ONLY전환가능여부',
 '카드론이용건수_누적',
 '카드론이용금액_누적',
 '쇼핑_온라인_이용금액',
 '일시상환론한도금액',
 '증감율_이용건수_할부_전월',
 '쇼핑_마트_이용금액',
 '연회비발생카드수_B0M',
 '컨택건수_이용유도_EM_B0M',
 '평잔_CA_3M',
 '월상환론한도금액',
 '증감율_이용금액_신용_분기',
 '증감율_이용건수_CA_전월',
 'RV_평균잔액_R3M',
 '인입월수_IB_R6M',
 '이용후경과월_체크',
 '이용건수_C페이_R6M',
 '여유_숙박이용금액',
 '회원여부_이용가능_카드론',
 'RP건수_보험_B0M',
 '탈회횟수_발급1년이내',
 '홈페이지_선결제건수_R6M',
 '교통_주유이용금액',
 '컨택건수_신용발급_TM_R6M',
 '할인금액_R3M',
 '동의여부_한도증액안내',
 '이용금액_해외',
 '청구금액_B0